In [1]:
import requests
import json
import requests
import wikipedia as wp
from pprint import pprint
import pandas as pd
g_key = ''




In [2]:
#https://en.wikipedia.org/wiki/List_of_hospitals_in_Georgia_(U.S._state)

def hospital_data():
    html = wp.page("List_of_hospitals_in_Georgia_(U.S._state)").html().encode("UTF-8")
    try: 
        df = pd.read_html(html)[1]  # Try 2nd table first as most pages contain contents table first
    except IndexError:
        df = pd.read_html(html)[0]
    return df


In [3]:
hospital_df= hospital_data()

hospital_df.tail()

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes
132,WellStar Paulding Hospital,Dallas,Paulding,216,NaN,NaN,1954,WellStar,NaN
133,WellStar Spalding Regional Hospital,Griffin,Spalding,160,NaN,NaN,NaN,WellStar,Formerly Spalding Regional Medical Center
134,WellStar Sylvan Grove Hospital,Jackson,Butts,21,NaN,NaN,NaN,WellStar,NaN
135,WellStar West Georgia Medical Center,LaGrange,Troup,542,Level III,NaN,1937,WellStar,NaN
136,Wills Memorial Hospital,Washington,Wilkes,25,NaN,NaN,1924,â,NaN


In [4]:
# url = "http://www.communitybenefitinsight.org/api/get_hospitals.php?"
# state= 'GA'
# query_url = f"{url}state={state}?"


# response = requests.get(query_url, headers={"User-Agent": "XY"}).json()

# pprint(len(response))
# #88

In [5]:
#Get Place ID of each hospital (https://developers.google.com/places/web-service/search)
hospital_df["Place ID"] = ''

params = {"keyword" : "Hospital",
          "key" : g_key,
          "inputtype" : "textquery"
         }


for index, row in hospital_df.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
        params["input"] = row["Hospital name"]
        response = requests.get(url, params=params).json()
        
        #Get Place ID
        place_id = response["candidates"][0]["place_id"]
        hospital_df.loc[index,"Place ID"] = place_id
    except:
        pass

        

In [6]:
hospital_df.head()

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID
0,AdventHealth Gordon,Calhoun,Gordon,69,NaN,NaN,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg
1,AdventHealth Murray,Chatsworth,Murray,29,NaN,NaN,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4
2,Archbold Memorial Hospital,Thomasville,Thomas,264,Level II,NaN,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ
3,Archbold - Brooks County Hospital,Quitman,Brooks,25,NaN,NaN,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM
4,Archbold - Grady General Hospital,Cairo,Grady,60,NaN,NaN,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE


In [7]:
#Use place_id (https://developers.google.com/places/web-service/details) to pull in remaining address information
hospital_df["Latitude"] = ''
hospital_df["Longtitude"] = ''
hospital_df["Street Address"] = ''
hospital_df["City"] = ''
hospital_df["County"] = ''
hospital_df["State"] = ''
hospital_df["Zip Code"] = ''
hospital_df["Full Address"] = ''


params = {"key" : g_key
         }


for index, row in hospital_df.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/details/json"
        params["place_id"] = row["Place ID"]
        response = requests.get(url, params=params).json()
        
        #Pulling new values
        lat = response['result']['geometry']['location']['lat']
        long = response['result']['geometry']['location']['lng']
        street_number = response['result']['address_components'][0]['long_name']
        street_name = response['result']['address_components'][1]['long_name']
        address = f"{street_number} {street_name}"

        city = response['result']['address_components'][2]['long_name']
        county = response['result']['address_components'][3]['long_name']
        state = response['result']['address_components'][4]['long_name']
        zip_code = response['result']['address_components'][6]['long_name']
        full_address = response['result']['formatted_address']
        
        #Putting new values in Hospital DF
        hospital_df.loc[index,"Latitude"] = lat
        hospital_df.loc[index,"Longtitude"] = long
        hospital_df.loc[index,"Street Address"] = address
        hospital_df.loc[index,"City"] = city
        hospital_df.loc[index,"County"] = county
        hospital_df.loc[index,"State"] = state
        hospital_df.loc[index,"Zip Code"] = zip_code
        hospital_df.loc[index,"Full Address"] = full_address
        
    except:
        pass

In [8]:
hospital_df

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,NaN,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.5102,-84.9263,1035 Red Bud Road Northeast,Georgia,30701,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,NaN,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.78,-84.783,707 Old Dalton Ellijay Road,Georgia,30705,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,Level II,NaN,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.8252,-83.9721,915 Gordon Avenue,Georgia,31792,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,NaN,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795,-83.5591,903 North Court Street,Georgia,31643,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,NaN,NaN,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.8638,-84.1991,1155 5th Street Southeast,Georgia,39828,"1155 5th St SE, Cairo, GA 39828, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,WellStar Paulding Hospital,Hiram,Paulding County,216,NaN,NaN,1954,WellStar,NaN,ChIJMzJn-How9YgRhhfE1kXnu30,33.9011,-84.7816,2518 Jimmy Lee Smith Parkway,Georgia,30141,"2518 Jimmy Lee Smith Pkwy, Hiram, GA 30141, USA"
133,WellStar Spalding Regional Hospital,Griffin,Spalding County,160,NaN,NaN,NaN,WellStar,Formerly Spalding Regional Medical Center,ChIJyexMgeBh9IgRxUd1HX227ns,33.2393,-84.2673,601 South 8th Street,Georgia,30224,"601 S 8th St, Griffin, GA 30224, USA"
134,WellStar Sylvan Grove Hospital,Jackson,Butts County,21,NaN,NaN,NaN,WellStar,NaN,ChIJB5o2d9JB9IgRr-4NjnGt52c,33.3038,-83.9778,1050 McDonough Road,Georgia,30233,"1050 McDonough Rd, Jackson, GA 30233, USA"
135,WellStar West Georgia Medical Center,LaGrange,Troup County,542,Level III,NaN,1937,WellStar,NaN,ChIJ40ZJ5vtWi4gRK6PnDb-BAPU,33.0309,-85.0586,1514 Vernon Road,Georgia,30240,"1514 Vernon Rd, LaGrange, GA 30240, USA"


In [9]:
#Strip out hospitals with correct data
correct_hospitals = hospital_df.loc[hospital_df["State"] == "Georgia"]
correct_hospitals

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,NaN,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.5102,-84.9263,1035 Red Bud Road Northeast,Georgia,30701,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,NaN,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.78,-84.783,707 Old Dalton Ellijay Road,Georgia,30705,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,Level II,NaN,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.8252,-83.9721,915 Gordon Avenue,Georgia,31792,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,NaN,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795,-83.5591,903 North Court Street,Georgia,31643,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,NaN,NaN,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.8638,-84.1991,1155 5th Street Southeast,Georgia,39828,"1155 5th St SE, Cairo, GA 39828, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,WellStar Paulding Hospital,Hiram,Paulding County,216,NaN,NaN,1954,WellStar,NaN,ChIJMzJn-How9YgRhhfE1kXnu30,33.9011,-84.7816,2518 Jimmy Lee Smith Parkway,Georgia,30141,"2518 Jimmy Lee Smith Pkwy, Hiram, GA 30141, USA"
133,WellStar Spalding Regional Hospital,Griffin,Spalding County,160,NaN,NaN,NaN,WellStar,Formerly Spalding Regional Medical Center,ChIJyexMgeBh9IgRxUd1HX227ns,33.2393,-84.2673,601 South 8th Street,Georgia,30224,"601 S 8th St, Griffin, GA 30224, USA"
134,WellStar Sylvan Grove Hospital,Jackson,Butts County,21,NaN,NaN,NaN,WellStar,NaN,ChIJB5o2d9JB9IgRr-4NjnGt52c,33.3038,-83.9778,1050 McDonough Road,Georgia,30233,"1050 McDonough Rd, Jackson, GA 30233, USA"
135,WellStar West Georgia Medical Center,LaGrange,Troup County,542,Level III,NaN,1937,WellStar,NaN,ChIJ40ZJ5vtWi4gRK6PnDb-BAPU,33.0309,-85.0586,1514 Vernon Road,Georgia,30240,"1514 Vernon Rd, LaGrange, GA 30240, USA"


In [10]:
#strip out hospitals with shifted/bad data
fix_hospitals = hospital_df.loc[hospital_df["State"] != "Georgia"]
fix_hospitals



,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
6,Augusta University Children's Hospital of Georgia,Medical District,Augusta,154,NaN,No,NaN,Augusta University,NaN,ChIJoyV_F0jM-YgRqAe60-Abauk,33.4711,-81.9878,1446 Harper Street,Richmond County,United States,"1446 Harper St, Augusta, GA 30912, USA"
7,Augusta University Medical Center,Medical District,Augusta,478,Level I,Yes,NaN,Augusta University,NaN,ChIJyX-W70nM-YgRtaRCmY78n_g,33.4713,-81.9879,1120 15th Street,Richmond County,United States,"1120 15th St, Augusta, GA 30912, USA"
12,Candler Hospital,Midtown,Savannah,384,NaN,NaN,1804,SJCHS,"Oldest hospital in Georgia, and second oldest ...",ChIJpb2fD5N1-4gRM-PXE90FV6c,32.0278,-81.1001,5353 Reynolds Street,Chatham County,United States,"5353 Reynolds St, Savannah, GA 31405, USA"
17,Children's Healthcare of Atlanta Hughes Spalding,Downtown Atlanta,Atlanta,NaN,NaN,Yes,1952,CHOA,NaN,ChIJE0RlP4kD9YgRf-FdJvSfWXw,33.7539,-84.382,35 Jesse Hill Junior Drive Southeast,Fulton County,United States,"35 Jesse Hill Jr Dr SE, Atlanta, GA 30303, USA"
18,Children's Healthcare of Atlanta Scottish Rite,Perimeter Center,Atlanta,NaN,Level II Peds,Yes,1915,CHOA,NaN,ChIJV-6vqjQJ9YgRhvQE1FyLn-g,33.9062,-84.3532,1001 Johnson Ferry Road Northeast,Fulton County,United States,"1001 Johnson Ferry Rd NE, Atlanta, GA 30342, USA"
30,Eastside Medical Center,Peachtree Parkway,Peachtree Corners,249,NaN,NaN,NaN,HCA,NaN,ChIJaTUBHwOh9YgRoHi7L2zoz4E,33.964,-84.2259,100 5707,Gwinnett County,United States,"5707 Peachtree Pkwy #100, Peachtree Corners, G..."
37,Emory Saint Joseph's Hospital,Perimeter Center,Atlanta,276,NaN,NaN,1880,Emory,Formerly Saint Joseph's Hospital of Atlanta,ChIJW3epPTQJ9YgRxf9a5U20qNg,33.9084,-84.3499,5665 Peachtree Dunwoody Road,Fulton County,United States,"5665 Peachtree Dunwoody Rd, Atlanta, GA 30342,..."
39,Emory University Hospital Midtown,SoNo,Atlanta,511,NaN,Yes,1908,Emory,Formerly Crawford W. Long Memorial Hospital,ChIJ42AWonoE9YgRonLLiSdYxtA,33.7687,-84.3864,550 Peachtree Street Northeast,Fulton County,United States,"550 Peachtree St NE, Atlanta, GA 30308, USA"
43,Fannin Regional Hospital,Old Highway 5,Blue Ridge,50,NaN,NaN,NaN,â,NaN,ChIJN7SfeV2vX4gRu5N9mgngcXo,34.9177,-84.3743,North 2855,Fannin County,United States,"2855 Old Hwy 5 North, Blue Ridge, GA 30513, USA"
47,Grady Memorial Hospital,Downtown Atlanta,Atlanta,961,Level I,Yes,1892,â,NaN,ChIJ6Rc8z44D9YgR28vCVjqbUGw,33.7518,-84.382,80 Jesse Hill Junior Drive Southeast,Fulton County,United States,"80 Jesse Hill Jr Dr SE, Atlanta, GA 30303, USA"


In [11]:
#Correct the bad address data for bad hospitals by shifting indices in the for loop
fix_hospitals["Latitude"] = ''
fix_hospitals["Longtitude"] = ''
fix_hospitals["Street Address"] = ''
fix_hospitals["City"] = ''
fix_hospitals["County"] = ''
fix_hospitals["State"] = ''
fix_hospitals["Zip Code"] = ''
fix_hospitals["Full Address"] = ''


params = {"key" : g_key
         }


for index, row in fix_hospitals.iterrows():
    try:
        url = "https://maps.googleapis.com/maps/api/place/details/json"
        params["place_id"] = row["Place ID"]
        response = requests.get(url, params=params).json()
        lat = response['result']['geometry']['location']['lat']
        long = response['result']['geometry']['location']['lng']
        street_number = response['result']['address_components'][0]['long_name']
        street_name = response['result']['address_components'][1]['long_name']
        address = f"{street_number} {street_name}"


        city = response['result']['address_components'][2]['long_name']
        county = response['result']['address_components'][3]['long_name']
        state = response['result']['address_components'][4]['long_name']
        zip_code = response['result']['address_components'][6]['long_name']
        full_address = response['result']['formatted_address']
        
        #Putting new values in Hospital DF
        fix_hospitals.loc[index,"Latitude"] = lat
        fix_hospitals.loc[index,"Longtitude"] = long
        fix_hospitals.loc[index,"Street Address"] = address
        fix_hospitals.loc[index,"City"] = city
        fix_hospitals.loc[index,"County"] = county
        fix_hospitals.loc[index,"State"] = state
        fix_hospitals.loc[index,"Zip Code"] = zip_code
        fix_hospitals.loc[index,"Full Address"] = full_address
        
    except:
        pass

C:\Users\kledesma\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kledesma\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kledesma\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [12]:
#Verifying data is fixed for fix_hospitals df
fix_hospitals.head()

,Hospital name,City,County,Acute Care Beds,Traumacenterdesignation,COTH[2]TeachingHospital,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
6,Augusta University Children's Hospital of Georgia,Medical District,Augusta,154,NaN,No,NaN,Augusta University,NaN,ChIJoyV_F0jM-YgRqAe60-Abauk,33.4711,-81.9878,1446 Harper Street,Richmond County,United States,"1446 Harper St, Augusta, GA 30912, USA"
7,Augusta University Medical Center,Medical District,Augusta,478,Level I,Yes,NaN,Augusta University,NaN,ChIJyX-W70nM-YgRtaRCmY78n_g,33.4713,-81.9879,1120 15th Street,Richmond County,United States,"1120 15th St, Augusta, GA 30912, USA"
12,Candler Hospital,Midtown,Savannah,384,NaN,NaN,1804,SJCHS,"Oldest hospital in Georgia, and second oldest ...",ChIJpb2fD5N1-4gRM-PXE90FV6c,32.0278,-81.1001,5353 Reynolds Street,Chatham County,United States,"5353 Reynolds St, Savannah, GA 31405, USA"
17,Children's Healthcare of Atlanta Hughes Spalding,Downtown Atlanta,Atlanta,NaN,NaN,Yes,1952,CHOA,NaN,ChIJE0RlP4kD9YgRf-FdJvSfWXw,33.7539,-84.382,35 Jesse Hill Junior Drive Southeast,Fulton County,United States,"35 Jesse Hill Jr Dr SE, Atlanta, GA 30303, USA"
18,Children's Healthcare of Atlanta Scottish Rite,Perimeter Center,Atlanta,NaN,Level II Peds,Yes,1915,CHOA,NaN,ChIJV-6vqjQJ9YgRhvQE1FyLn-g,33.9062,-84.3532,1001 Johnson Ferry Road Northeast,Fulton County,United States,"1001 Johnson Ferry Rd NE, Atlanta, GA 30342, USA"


In [13]:
#Concatenating the two DF's back together and printing, doing some checks
cleaned_hospitals = pd.concat([correct_hospitals, fix_hospitals])

cleaned_hospitals["State"].unique()
cleaned_hospitals["City"].unique()
cleaned_hospitals["County"].unique()

AttributeError: 'DataFrame' object has no attribute 'header'

In [15]:
#Re-ordering the columns, dropping irrelevant columns
del cleaned_hospitals['Traumacenterdesignation']
del cleaned_hospitals['COTH[2]TeachingHospital']

In [16]:
cleaned_hospitals.head()

,Hospital name,City,County,Acute Care Beds,YearFounded,Affiliation,Notes,Place ID,Latitude,Longtitude,Street Address,State,Zip Code,Full Address
0,AdventHealth Gordon,Calhoun,Gordon County,69,NaN,AdventHealth,Formerly Gordon Hospital,ChIJfVjpjer9X4gRGK7NwGG_tbg,34.5102,-84.9263,1035 Red Bud Road Northeast,Georgia,30701,"1035 Red Bud Rd NE, Calhoun, GA 30701, USA"
1,AdventHealth Murray,Chatsworth,Murray County,29,NaN,AdventHealth,Formerly Murray Medical Center,ChIJ-U0scjndX4gRG-FoXnNaHI4,34.78,-84.783,707 Old Dalton Ellijay Road,Georgia,30705,"707 Old Dalton Ellijay Rd, Chatsworth, GA 3070..."
2,Archbold Memorial Hospital,Thomasville,Thomas County,264,1925,Archbold,NaN,ChIJsxblg8q17YgR8XXHSBDuObQ,30.8252,-83.9721,915 Gordon Avenue,Georgia,31792,"915 Gordon Ave, Thomasville, GA 31792, USA"
3,Archbold - Brooks County Hospital,Quitman,Brooks County,25,NaN,Archbold,NaN,ChIJiyuklBp_7ogRmoGX5XNztMM,30.795,-83.5591,903 North Court Street,Georgia,31643,"903 N Court St, Quitman, GA 31643, USA"
4,Archbold - Grady General Hospital,Cairo,Grady County,60,1960,Archbold,NaN,ChIJL2eo1Vup7YgRbnlST-utsVE,30.8638,-84.1991,1155 5th Street Southeast,Georgia,39828,"1155 5th St SE, Cairo, GA 39828, USA"


In [19]:
gmaps.configure(api_key=g_key)

locations = cleaned_hospitals[['Latitude', 'Longtitude']]


#Creating heat layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity = 200,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig



AttributeError: 'NoneType' object has no attribute 'rename'